In [1]:
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
# Load the staff data
staff_data = pd.read_excel('staff.xlsx')

# Handle missing values in the 'Type' column (example: fill with 'Unknown')
staff_data['Type'] = staff_data['Type'].fillna('Unknown')

# Group and aggregate data for deployment plan calculations
grouped_data = staff_data.groupby(['Project', 'Job', 'Type', 'Month']).agg(
    Peak_Number=('Job', 'count'),
    Average_Start_Month=('Month', 'min'),
    Average_End_Month=('Month', 'max')
).reset_index()


In [3]:
def calculate_start_date(max_job_month):
    # Example logic: Start on the first day of the max_job_month
    start_date = datetime.date.today().replace(month=max_job_month, day=1)
    return start_date

def calculate_end_date(start_date, duration_months):
    # Example logic: End on the last day of the month containing start_date + duration_months
    end_date = (start_date + relativedelta(months=duration_months)).replace(day=31)
    return end_date

# Define the function to create the deployment plan for each job title
def create_deployment_plan_by_job_title(project_type, duration_months):

    # Filter data based on project type
    filtered_data = grouped_data[grouped_data['Type'] == project_type]

    # Create a list to store deployment plans for each job title
    deployment_plans = []

    for job_title, job_data in filtered_data.groupby(['Job', 'Month']):
        
        # Convert Month to datetime
        job_data['Month'] = pd.to_datetime(job_data['Month'])

        # Extract job title string from potential MultiIndex
        job_title_str = job_title[0]  # Assuming job title is the first element in index tuple

        # Find the month with the highest number of jobs for the job title
        max_job_month = job_data['Peak_Number'].idxmax()
        max_job_month = job_data.loc[max_job_month, 'Month'].month
        if not 1 <= max_job_month <= 12:
            raise ValueError("Invalid month value:", max_job_month)

        actual_job_count = job_data['Peak_Number'].loc[max_job_month]


        # Calculate start and end dates based on project-specific rules
        start_date = calculate_start_date(max_job_month)
        end_date = calculate_end_date(start_date, duration_months)

        # Check if a dictionary for this job title already exists in the list
        job_plan = next((plan for plan in deployment_plans if plan['Job_Title'] == job_title_str), None)

        # Create a new dictionary if it doesn't exist, otherwise update the existing one
        if job_plan is None:
            job_plan = {
                'Project_Type': project_type,
                'Job_Title': job_title_str,
                # ... (other fields)
            }
            deployment_plans.append(job_plan)

        # Update the dictionary with current month's information
        job_plan['Number_of_Jobs'] = max(job_plan.get('Number_of_Jobs', 0), actual_job_count)
        job_plan['Start_Date'] = start_date  # Assign dates only once
        job_plan['End_Date'] = end_date

    return deployment_plans

In [4]:
# Get user input
project_type = input("Enter the project type: ")
duration_months = int(input("Enter the project duration in months: "))

# Create and print the deployment plans for each job title
deployment_plans = create_deployment_plan_by_job_title(project_type, duration_months)
print("Deployment Plans for", project_type, "Project by Job Title:")
for plan in deployment_plans:
    print(plan)

ValueError: invalid literal for int() with base 10: ''

In [ ]:
# Create the Excel writer object before writing to the file
writer = pd.ExcelWriter('deployment_plans.xlsx')

# Create a single DataFrame to hold all deployment plans
all_plans_df = pd.DataFrame(deployment_plans)

# Write the DataFrame to a single worksheet in the Excel file
all_plans_df.to_excel(writer, sheet_name="Deployment Plans", index=False)

# Close the writer to save the Excel file
writer.close()

print("Deployment plans saved to deployment_plans.xlsx")

Deployment plans saved to deployment_plans.xlsx


In [ ]:
grouped_data

,Project,Job,Type,Month,Peak_Number,Average_Start_Month,Average_End_Month
0,10th of Ramadan Bridge,Blacksmith Supervisor,Bridges,2020-06-01,1,2020-06-01,2020-06-01
1,10th of Ramadan Bridge,Blacksmith Supervisor,Bridges,2020-07-01,1,2020-07-01,2020-07-01
2,10th of Ramadan Bridge,Blacksmith Supervisor,Bridges,2020-08-01,1,2020-08-01,2020-08-01
3,10th of Ramadan Bridge,Carpenter Supervisor,Bridges,2020-06-01,1,2020-06-01,2020-06-01
4,10th of Ramadan Bridge,Carpenter Supervisor,Bridges,2020-07-01,1,2020-07-01,2020-07-01
...,...,...,...,...,...,...,...
69696,Zed Towers Phase II,Technical Office Engineer,Residential,2022-10-01,1,2022-10-01,2022-10-01
69697,Zed Towers Phase II,Technical Office Engineer,Residential,2022-11-01,1,2022-11-01,2022-11-01
69698,Zed Towers Phase II,Technical Office Engineer,Residential,2022-12-01,1,2022-12-01,2022-12-01
69699,Zed Towers Phase II,Technical Office Engineer,Residential,2023-10-01,1,2023-10-01,2023-10-01
